# Procesamiento de información 

## Unidad 8. Actividad 8A: Desempeño de los clasificadores

#### Claudio Daniel Pacheco Castro

### Introducción<br>
El presente documento muestra el desarrollo de la descarga de información así como de la limpieza de la misma para poder contar con datos que, posteriormente, se puedan utilizar para la definición de un modelo de clasificación.
### Desarrollo de la actividad<br>
#### Importar las librerías necesarias

In [1]:

#Importar librerías
import pandas as pd
import os
import zipfile
import shutil

El primer paso consiste en definir el directorio de trabajo y, dentro de este, el de descarga de información.

In [2]:
#Definir directorio de trabajo
#Directorio de trabajo. Si existe directorio D:/, se selecciona si no, C:/
if os.path.exists('D:/'):
    os.chdir('D:/github/proyecto_infotec')
else:
    os.chdir('C:/users/claud/Documents/Github/proyecto_infotec')

In [79]:
#crear directorio de salida
if not os.path.exists('datos'):
    os.makedirs('datos')

### Descarga de información a nivel manzana

Una vez definido el directorio y creada la carpeta de descarga, se procede a descargarla la información del INEGI. Para este caso, se utilizan los datos del Censo de Población y Vivienda 2020 a nivel manzana.

In [80]:
#URL básica para descarga de archivos
url_basica="https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/ageb_mza_urbana_"

Con la URL básica, se obtiene la información de cada una de las entidades federativas:

In [81]:
#Descargar archivos y descomprimir archivos

for entidad in range(1,33):
    #Si existe el archivo, no se descarga
    if os.path.exists("datos/conjunto_de_datos_ageb_urbana_01_cpv2020.csv"):
        pass
    else:
        if entidad<10:
            url=url_basica+"0"+str(entidad)+"_cpv2020_csv.zip"
        else:
            url=url_basica+str(entidad)+"_cpv2020_csv.zip"
        
#Descargar archivo
        os.system("curl -o datos/entidad_"+str(entidad)+".zip "+url)
#Descomprimir archivo
        with zipfile.ZipFile("datos/entidad_"+str(entidad)+".zip","r") as zip_ref:
            zip_ref.extractall("datos")
#Eliminar archivo zip
        os.remove("datos/entidad_"+str(entidad)+".zip")

In [85]:
#Sacar de la carpeta conjunto_de_datos el archivo csv de cada entidad
for entidad in range(1,33):
    if entidad<10:
        shutil.move("datos/ageb_mza_urbana_0"+str(entidad)+"_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_0"+str(entidad)+"_cpv2020.csv","datos/conjunto_de_datos_ageb_urbana_0"+str(entidad)+"_cpv2020.csv")
    else:
        shutil.move("datos/ageb_mza_urbana_"+str(entidad)+"_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.csv","datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.csv")

In [88]:
#Eliminar las carpetas del directorio de trabajo si existen
for entidad in range(1,33):
    if entidad<10:
        if os.path.exists("datos/ageb_mza_urbana_"+"0"+str(entidad)+"_cpv2020"):
            shutil.rmtree("datos/ageb_mza_urbana_"+"0"+str(entidad)+"_cpv2020")
    else:
        if os.path.exists("datos/ageb_mza_urbana_"+str(entidad)+"_cpv2020"):
            shutil.rmtree("datos/ageb_mza_urbana_"+str(entidad)+"_cpv2020")

Al finalizar la descarga y descompresión de la información, se procede a unir los archivos en uno solo:

In [31]:
#Pegar todos los archivos csv en una sola data frame
df=pd.DataFrame()
for entidad in range(1,33):
    if entidad<10:
        df_temp=pd.read_csv("datos/conjunto_de_datos_ageb_urbana_"+"0"+str(entidad)+"_cpv2020.csv",encoding='utf-8', low_memory=False)
    else:
        df_temp=pd.read_csv("datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.csv",encoding='utf-8', low_memory=False)
    df=pd.concat([df,df_temp],axis=0)
    print("Se ha agregado la entidad "+str(entidad)," a la dataframe")

#Imprimir el número de registros con separadores de miles
print("Se concluyó la concatenación. El conjunto de datos cuenta con "+str(df.shape[1])+" columnas y "+str("{:,}".format(df.shape[0]))+" registros")
df.columns=df.columns.str.lower()
#Eliminar todos los archivos csv de la carpeta datos
#archivos=[x for x in os.listdir('datos') if x.endswith('.csv')]
#for archivo in archivos:
#    os.remove("datos/"+archivo)    

Se ha agregado la entidad 1  a la dataframe
Se ha agregado la entidad 2  a la dataframe
Se ha agregado la entidad 3  a la dataframe
Se ha agregado la entidad 4  a la dataframe
Se ha agregado la entidad 5  a la dataframe
Se ha agregado la entidad 6  a la dataframe
Se ha agregado la entidad 7  a la dataframe
Se ha agregado la entidad 8  a la dataframe
Se ha agregado la entidad 9  a la dataframe
Se ha agregado la entidad 10  a la dataframe
Se ha agregado la entidad 11  a la dataframe
Se ha agregado la entidad 12  a la dataframe
Se ha agregado la entidad 13  a la dataframe
Se ha agregado la entidad 14  a la dataframe
Se ha agregado la entidad 15  a la dataframe
Se ha agregado la entidad 16  a la dataframe
Se ha agregado la entidad 17  a la dataframe
Se ha agregado la entidad 18  a la dataframe
Se ha agregado la entidad 19  a la dataframe
Se ha agregado la entidad 20  a la dataframe
Se ha agregado la entidad 21  a la dataframe
Se ha agregado la entidad 22  a la dataframe
Se ha agregado la e

In [3]:
#Mostrar todos los nombres de las columnas
print(df.columns.tolist())

['entidad', 'nom_ent', 'mun', 'nom_mun', 'loc', 'nom_loc', 'ageb', 'mza', 'pobtot', 'pobfem', 'pobmas', 'p_0a2', 'p_0a2_f', 'p_0a2_m', 'p_3ymas', 'p_3ymas_f', 'p_3ymas_m', 'p_5ymas', 'p_5ymas_f', 'p_5ymas_m', 'p_12ymas', 'p_12ymas_f', 'p_12ymas_m', 'p_15ymas', 'p_15ymas_f', 'p_15ymas_m', 'p_18ymas', 'p_18ymas_f', 'p_18ymas_m', 'p_3a5', 'p_3a5_f', 'p_3a5_m', 'p_6a11', 'p_6a11_f', 'p_6a11_m', 'p_8a14', 'p_8a14_f', 'p_8a14_m', 'p_12a14', 'p_12a14_f', 'p_12a14_m', 'p_15a17', 'p_15a17_f', 'p_15a17_m', 'p_18a24', 'p_18a24_f', 'p_18a24_m', 'p_15a49_f', 'p_60ymas', 'p_60ymas_f', 'p_60ymas_m', 'rel_h_m', 'pob0_14', 'pob15_64', 'pob65_mas', 'prom_hnv', 'pnacent', 'pnacent_f', 'pnacent_m', 'pnacoe', 'pnacoe_f', 'pnacoe_m', 'pres2015', 'pres2015_f', 'pres2015_m', 'presoe15', 'presoe15_f', 'presoe15_m', 'p3ym_hli', 'p3ym_hli_f', 'p3ym_hli_m', 'p3hlinhe', 'p3hlinhe_f', 'p3hlinhe_m', 'p3hli_he', 'p3hli_he_f', 'p3hli_he_m', 'p5_hli', 'p5_hli_nhe', 'p5_hli_he', 'phog_ind', 'pob_afro', 'pob_afro_f', 'po

In [4]:
df

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,pobtot,pobfem,...,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
0,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la entidad,0000,0,1425607,728924,...,147818,359895,236003,174089,98724,70126,6021,15323,128996,1711
1,1,Aguascalientes,1,Aguascalientes,0,Total del municipio,0000,0,948990,486917,...,116647,251719,178619,130290,80951,56131,3299,7293,74227,731
2,1,Aguascalientes,1,Aguascalientes,1,Total de la localidad urbana,0000,0,863893,444725,...,112002,232793,169675,123670,77719,53589,2995,5984,63661,595
3,1,Aguascalientes,1,Aguascalientes,1,Total AGEB urbana,0017,0,2237,1137,...,11,625,189,352,46,74,15,23,391,*
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,170,87,...,*,53,13,33,5,5,0,*,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33839,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,18,2,*,...,*,*,*,*,*,*,*,*,*,*
33840,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,19,0,0,...,0,0,0,0,0,0,0,0,0,0
33841,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,20,0,0,...,0,0,0,0,0,0,0,0,0,0
33842,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,21,2,*,...,*,*,*,*,*,*,*,*,*,*


Dado que únicamente nos interesan las variables de vivienda, se procede a eliminar las demás:

In [32]:
df= df.iloc[:, list(range(8)) + list(range(177, len(df.columns)))]
print("El conjunto de datos cuenta con "+str(df.shape[1])+" columnas y "+str("{:,}".format(df.shape[0]))+" registros")
df

El conjunto de datos cuenta con 61 columnas y 1,683,504 registros


,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,tvivpar,vivpar_hab,vivparh_cv,tvivparhab,vivpar_des,vivpar_ut,ocupvivpar,prom_ocup,pro_ocup_c,vph_pisodt,vph_pisoti,vph_1dor,vph_2ymasd,vph_1cuart,vph_2cuart,vph_3ymasc,vph_c_elec,vph_s_elec,vph_aguadv,vph_aeasp,vph_aguafv,vph_tinaco,vph_cister,vph_excsa,vph_letr,vph_drenaj,vph_nodren,vph_c_serv,vph_ndeaed,vph_dsadma,vph_ndacmm,vph_snbien,vph_refri,vph_lavad,vph_hmicro,vph_autom,vph_moto,vph_bici,vph_radio,vph_tv,vph_pc,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
0,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la entidad,0000,0,463972,386671,453099,375798,386011,386445,60327,16974,1421200,3.68,0.89,382634,2899,86423,299141,6390,30686,348487,384361,1210,383430,355192,2137,295903,152521,382597,1011,383148,2379,380982,184,382121,130744,876,365189,335710,238571,240381,45642,105752,312002,370411,177149,147818,359895,236003,174089,98724,70126,6021,15323,128996,1711
1,1,Aguascalientes,1,Aguascalientes,0,Total del municipio,0000,0,313256,266942,304033,257719,266508,266778,37113,9201,945505,3.54,0.84,264662,1530,55437,210784,3185,17449,245585,265785,438,265232,254151,989,199994,122801,265261,520,265392,798,264378,53,264988,88348,380,254959,234193,176002,169694,26624,54393,221315,257061,136923,116647,251719,178619,130290,80951,56131,3299,7293,74227,731
2,1,Aguascalientes,1,Aguascalientes,1,Total de la localidad urbana,0000,0,286646,246259,278009,237622,245873,246116,33043,7344,862148,3.5,0.82,244472,1098,49862,195737,2478,15101,228019,245301,300,245236,238533,363,184683,114722,245424,60,245459,111,244955,19,245364,82142,313,235833,216609,165374,156488,22727,47846,204535,237274,130384,112002,232793,169675,123670,77719,53589,2995,5984,63661,595
3,1,Aguascalientes,1,Aguascalientes,1,Total AGEB urbana,0017,0,1288,648,1273,633,648,648,633,7,2237,3.45,1.14,647,*,192,456,*,38,609,645,3,648,511,0,35,21,648,0,648,0,645,0,648,451,*,514,416,210,141,72,95,560,575,141,11,625,189,352,46,74,15,23,391,*
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54,82,54,54,54,28,0,170,3.15,1.06,54,0,21,33,0,4,50,54,0,54,54,0,*,0,54,0,54,0,54,0,54,33,0,46,35,19,16,8,9,49,49,11,*,53,13,33,5,5,0,*,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33839,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,18,3,1,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
33840,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,19,1,*,*,*,*,*,*,*,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33841,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,20,0,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33842,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,21,1,1,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1683504 entries, 0 to 33843
Data columns (total 61 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   entidad      1683504 non-null  int64 
 1   nom_ent      1683504 non-null  object
 2   mun          1683504 non-null  int64 
 3   nom_mun      1683504 non-null  object
 4   loc          1683504 non-null  int64 
 5   nom_loc      1683504 non-null  object
 6   ageb         1683504 non-null  object
 7   mza          1683504 non-null  int64 
 8   vivtot       1683504 non-null  int64 
 9   tvivhab      1683504 non-null  object
 10  tvivpar      1683504 non-null  object
 11  vivpar_hab   1683504 non-null  object
 12  vivparh_cv   1683504 non-null  object
 13  tvivparhab   1683504 non-null  object
 14  vivpar_des   1683504 non-null  object
 15  vivpar_ut    1683504 non-null  object
 16  ocupvivpar   1683504 non-null  object
 17  prom_ocup    1683504 non-null  object
 18  pro_ocup_c   1683504 non

#### Eliminar agregados

Estos conjuntos de información cuentan con agregados por entidad, municipio, localidad y manzana. Para este caso, únicamente nos interesa la información a nivel manzana, por lo que se procede a eliminar los agregados:

In [34]:
#Eliminar agregados de entidad. 
df=df[df['mun']!=0]
#Eliminar agregados de municipio. 
df=df[df['loc']!=0]
#Eliminar agregados de localidad.
df=df[df['ageb']!='0000']
#Eliminar agregados de manzana.
df=df[df['mza']!=0]


In [35]:
df

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,tvivpar,vivpar_hab,vivparh_cv,tvivparhab,vivpar_des,vivpar_ut,ocupvivpar,prom_ocup,pro_ocup_c,vph_pisodt,vph_pisoti,vph_1dor,vph_2ymasd,vph_1cuart,vph_2cuart,vph_3ymasc,vph_c_elec,vph_s_elec,vph_aguadv,vph_aeasp,vph_aguafv,vph_tinaco,vph_cister,vph_excsa,vph_letr,vph_drenaj,vph_nodren,vph_c_serv,vph_ndeaed,vph_dsadma,vph_ndacmm,vph_snbien,vph_refri,vph_lavad,vph_hmicro,vph_autom,vph_moto,vph_bici,vph_radio,vph_tv,vph_pc,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54,82,54,54,54,28,0,170,3.15,1.06,54,0,21,33,0,4,50,54,0,54,54,0,*,0,54,0,54,0,54,0,54,33,0,46,35,19,16,8,9,49,49,11,*,53,13,33,5,5,0,*,36,0
5,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,2,83,52,81,50,52,52,31,0,198,3.81,1.23,52,0,13,39,0,*,51,52,0,52,51,0,*,0,52,0,52,0,52,0,52,37,0,37,30,18,12,4,9,47,48,11,*,52,14,31,*,5,0,0,31,0
6,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,3,84,55,83,54,55,55,27,*,198,3.6,1.2,55,0,15,40,0,4,51,54,*,55,34,0,8,*,55,0,55,0,54,0,55,41,0,45,40,21,12,*,7,35,47,10,*,53,14,30,5,5,4,*,35,0
7,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,4,84,57,83,56,57,57,27,0,202,3.54,1.18,57,0,14,43,0,*,56,57,0,57,52,0,0,0,57,0,57,0,57,0,57,40,0,49,37,19,16,3,*,49,50,12,*,55,20,34,9,5,*,*,33,0
8,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,5,68,48,67,47,48,48,20,0,157,3.27,1.05,48,0,18,30,*,5,42,48,0,48,48,0,4,0,48,0,48,0,48,0,48,27,0,41,35,16,14,9,12,41,42,10,0,45,21,24,5,6,*,3,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33839,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,18,3,1,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
33840,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,19,1,*,*,*,*,*,*,*,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33841,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,20,0,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33842,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,21,1,1,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*


In [36]:
print("El conjunto de datos cuenta con "+str(df.shape[1])+" columnas y "+str("{:,}".format(df.shape[0]))+" registros")


El conjunto de datos cuenta con 61 columnas y 1,611,448 registros


#### Transformación de variables a numérico

Para poder trabajar con los datos, es necesario que las variables sean numéricas. En algunos casos, por el nivel de desagregación, las variables cuentan con un "*". Estos se sustituirán por NA.

In [37]:
#transformar variables de la 8 en adelante a numéricas
df.iloc[:,8:]=df.iloc[:,8:].apply(pd.to_numeric, errors='coerce')

C:\Users\claud\AppData\Local\Temp\ipykernel_19724\743606727.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,8:]=df.iloc[:,8:].apply(pd.to_numeric, errors='coerce')


In [38]:
df

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,tvivpar,vivpar_hab,vivparh_cv,tvivparhab,vivpar_des,vivpar_ut,ocupvivpar,prom_ocup,pro_ocup_c,vph_pisodt,vph_pisoti,vph_1dor,vph_2ymasd,vph_1cuart,vph_2cuart,vph_3ymasc,vph_c_elec,vph_s_elec,vph_aguadv,vph_aeasp,vph_aguafv,vph_tinaco,vph_cister,vph_excsa,vph_letr,vph_drenaj,vph_nodren,vph_c_serv,vph_ndeaed,vph_dsadma,vph_ndacmm,vph_snbien,vph_refri,vph_lavad,vph_hmicro,vph_autom,vph_moto,vph_bici,vph_radio,vph_tv,vph_pc,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54.0,82.0,54.0,54.0,54.0,28.0,0.0,170.0,3.15,1.06,54.0,0.0,21.0,33.0,0.0,4.0,50.0,54.0,0.0,54.0,54.0,0.0,NaN,0.0,54.0,0.0,54.0,0.0,54.0,0.0,54.0,33.0,0.0,46.0,35.0,19.0,16.0,8.0,9.0,49.0,49.0,11.0,NaN,53.0,13.0,33.0,5.0,5.0,0.0,NaN,36.0,0.0
5,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,2,83,52.0,81.0,50.0,52.0,52.0,31.0,0.0,198.0,3.81,1.23,52.0,0.0,13.0,39.0,0.0,NaN,51.0,52.0,0.0,52.0,51.0,0.0,NaN,0.0,52.0,0.0,52.0,0.0,52.0,0.0,52.0,37.0,0.0,37.0,30.0,18.0,12.0,4.0,9.0,47.0,48.0,11.0,NaN,52.0,14.0,31.0,NaN,5.0,0.0,0.0,31.0,0.0
6,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,3,84,55.0,83.0,54.0,55.0,55.0,27.0,NaN,198.0,3.60,1.20,55.0,0.0,15.0,40.0,0.0,4.0,51.0,54.0,NaN,55.0,34.0,0.0,8.0,NaN,55.0,0.0,55.0,0.0,54.0,0.0,55.0,41.0,0.0,45.0,40.0,21.0,12.0,NaN,7.0,35.0,47.0,10.0,NaN,53.0,14.0,30.0,5.0,5.0,4.0,NaN,35.0,0.0
7,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,4,84,57.0,83.0,56.0,57.0,57.0,27.0,0.0,202.0,3.54,1.18,57.0,0.0,14.0,43.0,0.0,NaN,56.0,57.0,0.0,57.0,52.0,0.0,0.0,0.0,57.0,0.0,57.0,0.0,57.0,0.0,57.0,40.0,0.0,49.0,37.0,19.0,16.0,3.0,NaN,49.0,50.0,12.0,NaN,55.0,20.0,34.0,9.0,5.0,NaN,NaN,33.0,0.0
8,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,5,68,48.0,67.0,47.0,48.0,48.0,20.0,0.0,157.0,3.27,1.05,48.0,0.0,18.0,30.0,NaN,5.0,42.0,48.0,0.0,48.0,48.0,0.0,4.0,0.0,48.0,0.0,48.0,0.0,48.0,0.0,48.0,27.0,0.0,41.0,35.0,16.0,14.0,9.0,12.0,41.0,42.0,10.0,0.0,45.0,21.0,24.0,5.0,6.0,NaN,3.0,22.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33839,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,18,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33840,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33841,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33842,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,21,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Eliminar información de manzanas que no tienen viviendas o que son NAs


In [39]:
#Eliminar manzanas sin viviendas o con NaN
df=df[df['tvivparhab']!=0]
df=df[df['tvivparhab'].notna()]
df

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,tvivpar,vivpar_hab,vivparh_cv,tvivparhab,vivpar_des,vivpar_ut,ocupvivpar,prom_ocup,pro_ocup_c,vph_pisodt,vph_pisoti,vph_1dor,vph_2ymasd,vph_1cuart,vph_2cuart,vph_3ymasc,vph_c_elec,vph_s_elec,vph_aguadv,vph_aeasp,vph_aguafv,vph_tinaco,vph_cister,vph_excsa,vph_letr,vph_drenaj,vph_nodren,vph_c_serv,vph_ndeaed,vph_dsadma,vph_ndacmm,vph_snbien,vph_refri,vph_lavad,vph_hmicro,vph_autom,vph_moto,vph_bici,vph_radio,vph_tv,vph_pc,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54.0,82.0,54.0,54.0,54.0,28.0,0.0,170.0,3.15,1.06,54.0,0.0,21.0,33.0,0.0,4.0,50.0,54.0,0.0,54.0,54.0,0.0,NaN,0.0,54.0,0.0,54.0,0.0,54.0,0.0,54.0,33.0,0.0,46.0,35.0,19.0,16.0,8.0,9.0,49.0,49.0,11.0,NaN,53.0,13.0,33.0,5.0,5.0,0.0,NaN,36.0,0.0
5,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,2,83,52.0,81.0,50.0,52.0,52.0,31.0,0.0,198.0,3.81,1.23,52.0,0.0,13.0,39.0,0.0,NaN,51.0,52.0,0.0,52.0,51.0,0.0,NaN,0.0,52.0,0.0,52.0,0.0,52.0,0.0,52.0,37.0,0.0,37.0,30.0,18.0,12.0,4.0,9.0,47.0,48.0,11.0,NaN,52.0,14.0,31.0,NaN,5.0,0.0,0.0,31.0,0.0
6,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,3,84,55.0,83.0,54.0,55.0,55.0,27.0,NaN,198.0,3.60,1.20,55.0,0.0,15.0,40.0,0.0,4.0,51.0,54.0,NaN,55.0,34.0,0.0,8.0,NaN,55.0,0.0,55.0,0.0,54.0,0.0,55.0,41.0,0.0,45.0,40.0,21.0,12.0,NaN,7.0,35.0,47.0,10.0,NaN,53.0,14.0,30.0,5.0,5.0,4.0,NaN,35.0,0.0
7,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,4,84,57.0,83.0,56.0,57.0,57.0,27.0,0.0,202.0,3.54,1.18,57.0,0.0,14.0,43.0,0.0,NaN,56.0,57.0,0.0,57.0,52.0,0.0,0.0,0.0,57.0,0.0,57.0,0.0,57.0,0.0,57.0,40.0,0.0,49.0,37.0,19.0,16.0,3.0,NaN,49.0,50.0,12.0,NaN,55.0,20.0,34.0,9.0,5.0,NaN,NaN,33.0,0.0
8,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,5,68,48.0,67.0,47.0,48.0,48.0,20.0,0.0,157.0,3.27,1.05,48.0,0.0,18.0,30.0,NaN,5.0,42.0,48.0,0.0,48.0,48.0,0.0,4.0,0.0,48.0,0.0,48.0,0.0,48.0,0.0,48.0,27.0,0.0,41.0,35.0,16.0,14.0,9.0,12.0,41.0,42.0,10.0,0.0,45.0,21.0,24.0,5.0,6.0,NaN,3.0,22.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33828,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,7,8,6.0,8.0,6.0,6.0,6.0,NaN,NaN,21.0,3.50,0.81,6.0,0.0,0.0,6.0,0.0,0.0,6.0,6.0,0.0,6.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,0.0,6.0,6.0,4.0,6.0,NaN,0.0,3.0,5.0,NaN,NaN,6.0,NaN,NaN,NaN,0.0,0.0,0.0,4.0,0.0
33831,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,10,6,4.0,6.0,4.0,4.0,4.0,NaN,0.0,15.0,3.75,0.88,4.0,0.0,0.0,4.0,0.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,NaN,0.0,4.0,4.0,NaN,3.0,NaN,0.0,4.0,3.0,NaN,NaN,4.0,4.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
33834,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,13,13,11.0,13.0,11.0,11.0,11.0,NaN,0.0,40.0,3.64,0.95,11.0,0.0,3.0,8.0,0.0,NaN,10.0,11.0,0.0,11.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,5.0,0.0,11.0,10.0,6.0,6.0,NaN,3.0,6.0,11.0,NaN,NaN,11.0,NaN,8.0,0.0,0.0,0.0,0.0,9.0,0.0
33838,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,17,5,3.0,5.0,3.0,3.0,3.0,NaN,0.0,11.0,3.67,1.00,3.0,0.0,NaN,NaN,0.0,0.0,3.0,3.0,0.0,3.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,NaN,0.0,3.0,3.0,NaN,NaN,NaN,0.0,NaN,3.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,3.0,0.0


#### Construcción de la clave geoestadística

Para poder realizar el vínculo información geográfica, se debe construir la clave geoestadística cocatenando la clave de entidad, la de municipio, la de localidad, ageb y manzana. Al final, se debe obtener una variable de tipo string con una longitud de 16 caracteres.

In [40]:
df['cvegeo'] = df['entidad'].astype(str).str.zfill(2) + df['mun'].astype(str).str.zfill(3) + df['loc'].astype(str).str.zfill(4) + df['ageb'].astype(str).str.zfill(4) + df['mza'].astype(str).str.zfill(3)

In [41]:
df

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,tvivpar,vivpar_hab,vivparh_cv,tvivparhab,vivpar_des,vivpar_ut,ocupvivpar,prom_ocup,pro_ocup_c,vph_pisodt,vph_pisoti,vph_1dor,vph_2ymasd,vph_1cuart,vph_2cuart,vph_3ymasc,vph_c_elec,vph_s_elec,vph_aguadv,vph_aeasp,vph_aguafv,vph_tinaco,vph_cister,vph_excsa,vph_letr,vph_drenaj,vph_nodren,vph_c_serv,vph_ndeaed,vph_dsadma,vph_ndacmm,vph_snbien,vph_refri,vph_lavad,vph_hmicro,vph_autom,vph_moto,vph_bici,vph_radio,vph_tv,vph_pc,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,cvegeo
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54.0,82.0,54.0,54.0,54.0,28.0,0.0,170.0,3.15,1.06,54.0,0.0,21.0,33.0,0.0,4.0,50.0,54.0,0.0,54.0,54.0,0.0,NaN,0.0,54.0,0.0,54.0,0.0,54.0,0.0,54.0,33.0,0.0,46.0,35.0,19.0,16.0,8.0,9.0,49.0,49.0,11.0,NaN,53.0,13.0,33.0,5.0,5.0,0.0,NaN,36.0,0.0,0100100010017001
5,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,2,83,52.0,81.0,50.0,52.0,52.0,31.0,0.0,198.0,3.81,1.23,52.0,0.0,13.0,39.0,0.0,NaN,51.0,52.0,0.0,52.0,51.0,0.0,NaN,0.0,52.0,0.0,52.0,0.0,52.0,0.0,52.0,37.0,0.0,37.0,30.0,18.0,12.0,4.0,9.0,47.0,48.0,11.0,NaN,52.0,14.0,31.0,NaN,5.0,0.0,0.0,31.0,0.0,0100100010017002
6,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,3,84,55.0,83.0,54.0,55.0,55.0,27.0,NaN,198.0,3.60,1.20,55.0,0.0,15.0,40.0,0.0,4.0,51.0,54.0,NaN,55.0,34.0,0.0,8.0,NaN,55.0,0.0,55.0,0.0,54.0,0.0,55.0,41.0,0.0,45.0,40.0,21.0,12.0,NaN,7.0,35.0,47.0,10.0,NaN,53.0,14.0,30.0,5.0,5.0,4.0,NaN,35.0,0.0,0100100010017003
7,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,4,84,57.0,83.0,56.0,57.0,57.0,27.0,0.0,202.0,3.54,1.18,57.0,0.0,14.0,43.0,0.0,NaN,56.0,57.0,0.0,57.0,52.0,0.0,0.0,0.0,57.0,0.0,57.0,0.0,57.0,0.0,57.0,40.0,0.0,49.0,37.0,19.0,16.0,3.0,NaN,49.0,50.0,12.0,NaN,55.0,20.0,34.0,9.0,5.0,NaN,NaN,33.0,0.0,0100100010017004
8,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,5,68,48.0,67.0,47.0,48.0,48.0,20.0,0.0,157.0,3.27,1.05,48.0,0.0,18.0,30.0,NaN,5.0,42.0,48.0,0.0,48.0,48.0,0.0,4.0,0.0,48.0,0.0,48.0,0.0,48.0,0.0,48.0,27.0,0.0,41.0,35.0,16.0,14.0,9.0,12.0,41.0,42.0,10.0,0.0,45.0,21.0,24.0,5.0,6.0,NaN,3.0,22.0,0.0,0100100010017005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33828,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,7,8,6.0,8.0,6.0,6.0,6.0,NaN,NaN,21.0,3.50,0.81,6.0,0.0,0.0,6.0,0.0,0.0,6.0,6.0,0.0,6.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,0.0,6.0,6.0,4.0,6.0,NaN,0.0,3.0,5.0,NaN,NaN,6.0,NaN,NaN,NaN,0.0,0.0,0.0,4.0,0.0,3205800010123007
33831,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,10,6,4.0,6.0,4.0,4.0,4.0,NaN,0.0,15.0,3.75,0.88,4.0,0.0,0.0,4.0,0.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,NaN,0.0,4.0,4.0,NaN,3.0,NaN,0.0,4.0,3.0,NaN,NaN,4.0,4.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3205800010123010
33834,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,13,13,11.0,13.0,11.0,11.0,11.0,NaN,0.0,40.0,3.64,0.95,11.0,0.0,3.0,8.0,0.0,NaN,10.0,11.0,0.0,11.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,5.0,0.0,11.0,10.0,6.0,6.0,NaN,3.0,6.0,11.0,NaN,NaN,11.0,NaN,8.0,0.0,0.0,0.0,0.0,9.0,0.0,3205800010123013
33838,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,17,5,3.0,5.0,3.0,3.0,3.0,NaN,0.0,11.0,3.67,1.00,3.0,0.0,NaN,NaN,0.0,0.0,3.0,3.0,0.0,3.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,NaN,0.0,3.0,3.0,NaN,NaN,NaN,0.0,NaN,3.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,3.0,0.0,3205800010123017


In [42]:
#Colocar la variable cvegeo al principio
df=df[['cvegeo']+df.columns.tolist()[:-1]]

In [43]:
#Mostrar todas las columnas
pd.set_option('display.max_columns', None)
df

,cvegeo,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,tvivpar,vivpar_hab,vivparh_cv,tvivparhab,vivpar_des,vivpar_ut,ocupvivpar,prom_ocup,pro_ocup_c,vph_pisodt,vph_pisoti,vph_1dor,vph_2ymasd,vph_1cuart,vph_2cuart,vph_3ymasc,vph_c_elec,vph_s_elec,vph_aguadv,vph_aeasp,vph_aguafv,vph_tinaco,vph_cister,vph_excsa,vph_letr,vph_drenaj,vph_nodren,vph_c_serv,vph_ndeaed,vph_dsadma,vph_ndacmm,vph_snbien,vph_refri,vph_lavad,vph_hmicro,vph_autom,vph_moto,vph_bici,vph_radio,vph_tv,vph_pc,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
4,0100100010017001,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54.0,82.0,54.0,54.0,54.0,28.0,0.0,170.0,3.15,1.06,54.0,0.0,21.0,33.0,0.0,4.0,50.0,54.0,0.0,54.0,54.0,0.0,NaN,0.0,54.0,0.0,54.0,0.0,54.0,0.0,54.0,33.0,0.0,46.0,35.0,19.0,16.0,8.0,9.0,49.0,49.0,11.0,NaN,53.0,13.0,33.0,5.0,5.0,0.0,NaN,36.0,0.0
5,0100100010017002,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,2,83,52.0,81.0,50.0,52.0,52.0,31.0,0.0,198.0,3.81,1.23,52.0,0.0,13.0,39.0,0.0,NaN,51.0,52.0,0.0,52.0,51.0,0.0,NaN,0.0,52.0,0.0,52.0,0.0,52.0,0.0,52.0,37.0,0.0,37.0,30.0,18.0,12.0,4.0,9.0,47.0,48.0,11.0,NaN,52.0,14.0,31.0,NaN,5.0,0.0,0.0,31.0,0.0
6,0100100010017003,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,3,84,55.0,83.0,54.0,55.0,55.0,27.0,NaN,198.0,3.60,1.20,55.0,0.0,15.0,40.0,0.0,4.0,51.0,54.0,NaN,55.0,34.0,0.0,8.0,NaN,55.0,0.0,55.0,0.0,54.0,0.0,55.0,41.0,0.0,45.0,40.0,21.0,12.0,NaN,7.0,35.0,47.0,10.0,NaN,53.0,14.0,30.0,5.0,5.0,4.0,NaN,35.0,0.0
7,0100100010017004,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,4,84,57.0,83.0,56.0,57.0,57.0,27.0,0.0,202.0,3.54,1.18,57.0,0.0,14.0,43.0,0.0,NaN,56.0,57.0,0.0,57.0,52.0,0.0,0.0,0.0,57.0,0.0,57.0,0.0,57.0,0.0,57.0,40.0,0.0,49.0,37.0,19.0,16.0,3.0,NaN,49.0,50.0,12.0,NaN,55.0,20.0,34.0,9.0,5.0,NaN,NaN,33.0,0.0
8,0100100010017005,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,5,68,48.0,67.0,47.0,48.0,48.0,20.0,0.0,157.0,3.27,1.05,48.0,0.0,18.0,30.0,NaN,5.0,42.0,48.0,0.0,48.0,48.0,0.0,4.0,0.0,48.0,0.0,48.0,0.0,48.0,0.0,48.0,27.0,0.0,41.0,35.0,16.0,14.0,9.0,12.0,41.0,42.0,10.0,0.0,45.0,21.0,24.0,5.0,6.0,NaN,3.0,22.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33828,3205800010123007,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,7,8,6.0,8.0,6.0,6.0,6.0,NaN,NaN,21.0,3.50,0.81,6.0,0.0,0.0,6.0,0.0,0.0,6.0,6.0,0.0,6.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,0.0,6.0,6.0,4.0,6.0,NaN,0.0,3.0,5.0,NaN,NaN,6.0,NaN,NaN,NaN,0.0,0.0,0.0,4.0,0.0
33831,3205800010123010,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,10,6,4.0,6.0,4.0,4.0,4.0,NaN,0.0,15.0,3.75,0.88,4.0,0.0,0.0,4.0,0.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,NaN,0.0,4.0,4.0,NaN,3.0,NaN,0.0,4.0,3.0,NaN,NaN,4.0,4.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
33834,3205800010123013,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,13,13,11.0,13.0,11.0,11.0,11.0,NaN,0.0,40.0,3.64,0.95,11.0,0.0,3.0,8.0,0.0,NaN,10.0,11.0,0.0,11.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0,5.0,0.0,11.0,10.0,6.0,6.0,NaN,3.0,6.0,11.0,NaN,NaN,11.0,NaN,8.0,0.0,0.0,0.0,0.0,9.0,0.0
33838,3205800010123017,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,17,5,3.0,5.0,3.0,3.0,3.0,NaN,0.0,11.0,3.67,1.00,3.0,0.0,NaN,NaN,0.0,0.0,3.0,3.0,0.0,3.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,NaN,0.0,3.0,3.0,NaN,NaN,NaN,0.0,NaN,3.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,3.0,0.0


Se realiza la comprobación de la longitud de la variable:

In [44]:
#Obtener resumen de largo de cvegeo
df['cvegeo'].str.len().value_counts()

16    1305275
Name: cvegeo, dtype: int64

#### Cálculo de porcentajes respecto a tvivparhab

In [51]:
#Dividir todas las variables entre tvivparhab y multiplicar por 100

ejemplo.iloc[:,20:]=ejemplo.iloc[:,20:].div(ejemplo['tvivparhab'],axis=0).multiply(100)

In [52]:
ejemplo

,cvegeo,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,tvivpar,vivpar_hab,vivparh_cv,tvivparhab,vivpar_des,vivpar_ut,ocupvivpar,prom_ocup,pro_ocup_c,vph_pisodt,vph_pisoti,vph_1dor,vph_2ymasd,vph_1cuart,vph_2cuart,vph_3ymasc,vph_c_elec,vph_s_elec,vph_aguadv,vph_aeasp,vph_aguafv,vph_tinaco,vph_cister,vph_excsa,vph_letr,vph_drenaj,vph_nodren,vph_c_serv,vph_ndeaed,vph_dsadma,vph_ndacmm,vph_snbien,vph_refri,vph_lavad,vph_hmicro,vph_autom,vph_moto,vph_bici,vph_radio,vph_tv,vph_pc,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
4,0100100010017001,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54.0,82.0,54.0,54.0,54.0,28.0,0.0,170.0,3.15,1.06,100.0,0.0,38.888889,61.111111,0.0,7.407407,92.592593,100.000000,0.0,100.0,100.000000,0.0,NaN,0.0,100.0,0.0,100.0,0.0,100.000000,0.0,100.0,61.111111,0.0,85.185185,64.814815,35.185185,29.629630,14.814815,16.666667,90.740741,90.740741,20.370370,NaN,98.148148,24.074074,61.111111,9.259259,9.259259,0.000000,NaN,66.666667,0.0
5,0100100010017002,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,2,83,52.0,81.0,50.0,52.0,52.0,31.0,0.0,198.0,3.81,1.23,100.0,0.0,25.000000,75.000000,0.0,NaN,98.076923,100.000000,0.0,100.0,98.076923,0.0,NaN,0.0,100.0,0.0,100.0,0.0,100.000000,0.0,100.0,71.153846,0.0,71.153846,57.692308,34.615385,23.076923,7.692308,17.307692,90.384615,92.307692,21.153846,NaN,100.000000,26.923077,59.615385,NaN,9.615385,0.000000,0.00,59.615385,0.0
6,0100100010017003,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,3,84,55.0,83.0,54.0,55.0,55.0,27.0,NaN,198.0,3.60,1.20,100.0,0.0,27.272727,72.727273,0.0,7.272727,92.727273,98.181818,NaN,100.0,61.818182,0.0,14.545455,NaN,100.0,0.0,100.0,0.0,98.181818,0.0,100.0,74.545455,0.0,81.818182,72.727273,38.181818,21.818182,NaN,12.727273,63.636364,85.454545,18.181818,NaN,96.363636,25.454545,54.545455,9.090909,9.090909,7.272727,NaN,63.636364,0.0
7,0100100010017004,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,4,84,57.0,83.0,56.0,57.0,57.0,27.0,0.0,202.0,3.54,1.18,100.0,0.0,24.561404,75.438596,0.0,NaN,98.245614,100.000000,0.0,100.0,91.228070,0.0,0.000000,0.0,100.0,0.0,100.0,0.0,100.000000,0.0,100.0,70.175439,0.0,85.964912,64.912281,33.333333,28.070175,5.263158,NaN,85.964912,87.719298,21.052632,NaN,96.491228,35.087719,59.649123,15.789474,8.771930,NaN,NaN,57.894737,0.0
8,0100100010017005,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,5,68,48.0,67.0,47.0,48.0,48.0,20.0,0.0,157.0,3.27,1.05,100.0,0.0,37.500000,62.500000,NaN,10.416667,87.500000,100.000000,0.0,100.0,100.000000,0.0,8.333333,0.0,100.0,0.0,100.0,0.0,100.000000,0.0,100.0,56.250000,0.0,85.416667,72.916667,33.333333,29.166667,18.750000,25.000000,85.416667,87.500000,20.833333,0.0,93.750000,43.750000,50.000000,10.416667,12.500000,NaN,6.25,45.833333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33828,3205800010123007,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,7,8,6.0,8.0,6.0,6.0,6.0,NaN,NaN,21.0,3.50,0.81,100.0,0.0,0.000000,100.000000,0.0,0.000000,100.000000,100.000000,0.0,100.0,100.000000,0.0,100.000000,0.0,100.0,0.0,100.0,0.0,100.000000,0.0,100.0,0.000000,0.0,100.000000,100.000000,66.666667,100.000000,NaN,0.000000,50.000000,83.333333,NaN,NaN,100.000000,NaN,NaN,NaN,0.000000,0.000000,0.00,66.666667,0.0
33831,3205800010123010,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,10,6,4.0,6.0,4.0,4.0,4.0,NaN,0.0,15.0,3.75,0.88,100.0,0.0,0.000000,100.000000,0.0,0.000000,100.000000,100.000000,0.0,100.0,100.000000,0.0,100.000000,0.0,100.0,0.0,100.0,0.0,100.000000,0.0,100.0,NaN,0.0,100.000000,100.000000,NaN,75.000000,NaN,0.000000,100.000000,75.000000,NaN,NaN,100.000000,100.000000,0.000000,NaN,0.000000,0.000000,0.00,0.000000,0.0
33834,3205800010123013,32,Zacatecas,

In [60]:
#Obtener máximos y mínimos de cada variable y ponerlos en un dataframe
maximos=ejemplo.iloc[:,20:].max()
minimos=ejemplo.iloc[:,20:].min()
#Crear un dataframe con los máximos y mínimos
maximos_minimos=pd.concat([minimos,maximos],axis=1)
#Renombrar columnas
maximos_minimos.columns=['minimo','maximo']
maximos_minimos

,minimo,maximo
vph_pisodt,0.0,100.0
vph_pisoti,0.0,100.0
vph_1dor,0.0,100.0
vph_2ymasd,0.0,100.0
vph_1cuart,0.0,100.0
vph_2cuart,0.0,100.0
vph_3ymasc,0.0,100.0
vph_c_elec,0.0,100.0
vph_s_elec,0.0,100.0
vph_aguadv,0.0,100.0
